# Classifying MNIST using random fourier features

We want to solve 
$$\min_{\theta}\mathbb{E}[y(X)- f(X;\theta)] + \lambda\vert\vert\theta\vert\vert$$
where $X$ is a handwritten digit and $y(X)$ is a one-hot vector containing the correct label. This problem has been approach with many different techniques and perhaps with highest precision when using CNNs. In this notebook we will formulate a "neural network" but with complex activation functions such that we are searching for the complex parameters $\theta \in \mathbf{C}^{10\times K}$.

We formulate the function as 

$$f_\theta(x;\theta) = \begin{bmatrix}\sum_{k=1}^K \theta_i^{(1)}s(\omega_k \cdot x) & \sum_{k=1}^K \theta_i^{(2)}s(\omega_k \cdot x) & \dots & \sum_{k=1}^K \theta_i^{(10)}s(\omega_k \cdot x) \end{bmatrix}^T \in \mathbb{C}^{10}$$

where $K$ is the number of "nodes" in the network and $s(x) = e^{ix}$. If we instead write this using matrix notation we get

$$f_\theta(x;\theta) = \begin{bmatrix} 
\theta_1^{(1)} & \theta_2^{(1)} & \dots & \theta_K^{(1)} \\ 
\theta_1^{(2)} & \theta_2^{(2)} & \dots & \theta_K^{(2)} \\ 
\vdots & \vdots  &  & \vdots \\ 
\theta_1^{(10)} & \theta_2^{(10)} & \dots & \theta_K^{(10)} \\ 
\end{bmatrix}  \begin{bmatrix} s(\omega_1 \cdot x) \\ s(\omega_2 \cdot x) \\\vdots \\ s(\omega_K \cdot x)
\end{bmatrix} = \Theta S $$

Then we have the system we want to solve the least squares problem 
$$\min_\Theta \vert\vert y(X) - \Theta S \vert\vert_2^2 + \lambda \vert\vert \Theta \vert \vert$$

which has the solution

$$\Theta = Y S^H(SS^H + \lambda I)^{-1}.$$

Let us implement this

In [1]:
import numpy as np
from keras.datasets import mnist

Define parameters and the activation function plus a mapping from label to one-hot vectors. 

In [51]:
K = 3000                 # Number of "nodes"
sigma = 0.0005           # frequency spread
reg_param = 2            # L2-reg
J = 50000                # Batch size

def s(x):
    return np.exp(1j*x)
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

Import the mnist data set and reshape and convert to one-hot. 

In [52]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
N = train_X.shape[0]
M = train_X.shape[1]**2
train_X = train_X.reshape(N,M)
test_X = test_X.reshape(test_X.shape[0],M)
train_Y = one_hot(train_y, 10)
test_Y = one_hot(test_y, 10)

In [53]:
# Extract a batch
X = train_X[0:J]
Y = train_Y[0:J].T

Calculate the $S$ matrix from the training and test set

In [54]:
omega = np.random.normal(0,sigma,(M,K))
S = s(np.dot(X, omega)).T
S_test = s(np.dot(test_X, omega)).T

Solve least squares problem with numpy. You could probably do this with some built in function.

In [55]:
Theta =  (Y @ S.conj().T) @ np.linalg.inv(S@ S.conj().T + reg_param*np.identity(K))

Now we can use $\Theta$ to make predictions by calulating $\Theta S$ and taking taking $max(Re(\Theta S))$ as the prediction

In [56]:
f_insample = Theta@S
f_outsample = Theta@S_test
train_pred = (np.argmax(np.real(f_insample),axis=0) == train_y[0:J]).sum()/J
test_pred = (np.argmax(np.real(f_outsample),axis=0) == test_y).sum()/10000

In [57]:
print('Correctly predicted samples on the training set: ', train_pred)
print('Correctly predicted samples on the test set: ', test_pred)

Correctly predicted samples on the training set:  0.9776
Correctly predicted samples on the test set:  0.9685


The results are not amazing, but still about 97% out-of-sample accuracy with a linear system which can be solved analytically and easily computed numerically. We could probably improve by finding the optimal frequencies or searching the the optimal distribution of frequencies. 